# Notebook For Q&A OffLine Evaluation - Part 1

## Step1 : create set of possible questions for each Q&A (aka Ground truth)

In [5]:
import os

from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq 
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

from langchain_groq import ChatGroq 

### loading the q&a vector database

In [6]:
# File paths and other configurations
EMBEDDING_MODEL =  os.getenv("EMBEDDING_MODEL")
LLM_MODEL = os.getenv("LLM_MODEL")
INDEX_NAME = os.getenv("INDEX_NAME")

embed_model = FastEmbedEmbeddings(model_name=EMBEDDING_MODEL)

new_vector_store = FAISS.load_local(
    INDEX_NAME, embed_model, allow_dangerous_deserialization=True
)

# Access the document store
stored_docs = new_vector_store.docstore._dict

Fetching 5 files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 49344.75it/s]


### a prompt for emulating test questions

In [7]:
# File paths and other configurations
LLM_MODEL = os.getenv("LLM_MODEL")

# Initialize the Groq LLM
groq_llm = ChatGroq(
    model=LLM_MODEL,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

emulation_prompt = """You emulate a potential client who is considering hypnotherapy services.
Formulate 5 questions this client might ask based on a FAQ record that is provided.
The record should contain the answer to the questions, and the questions should be complete and not too short.

The record:

question: {question}
answer: {text}

Provide the output as a valid JSON array without using code blocks, extra characters like asterisks (*), or numbering. The format should be exactly:

["question1", "question2", ..., "question5"]"""



emulation_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question","text"], template=emulation_prompt)
)

messages = [emulation_system_prompt]

emulation_tamplate = ChatPromptTemplate(
    input_variables=["question","text"], messages=messages
)

output_parser = StrOutputParser()

emulation_chain = emulation_tamplate | groq_llm | output_parser


### testing the prompt

In [8]:
answer = """Everyone is capable of change. Whether you will effect that change is largely down to you. Cognitive Hypnotherapy isn’t a miracle pill.
It’s a process of mental reprogramming. Our sessions will guide you to learn how to think, behave, and feel differently. 

Since I began practicing as a therapist, I’ve noticed some common traits among the clients who have made the most positive changes in their lives.
They are curious, willing to learn about themselves, and being prepared to put in the work. I can guide you. 
And I will support you always. But nobody can do it for you."""

reply = emulation_chain.invoke(dict(question='Q: Will hypnotherapy work for Me?',text=answer))
print(reply)

["Will hypnotherapy work for me even if I'm skeptical about its effectiveness?", 
"How much control will I have during the hypnotherapy sessions?", 
"What is expected from me in terms of effort and commitment for the therapy to be successful?", 
"Can cognitive hypnotherapy help me overcome habits that I've had for many years?", 
"What are the common traits of clients who have achieved positive changes through hypnotherapy?"]


### creating the ground truth dataset for testing the chatbot

In [17]:
import ast
from tqdm import tqdm  # Import tqdm

val_dict = dict(doc_id=[], orig_quest=[], questions=[])

# Iterate over the documents with a progress bar
for _, doc in tqdm(stored_docs.items(), desc="Processing Documents"):

    quest, answer = doc.page_content.split('\n', 1)
    doc_id = doc.metadata.get('doc_id', None)
    reply = emulation_chain.invoke(dict(question=quest, text=answer))

    try:
        # Try to convert the string to an actual list
        actual_list = ast.literal_eval(reply)
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing the string: {e}")
        continue

    val_dict['doc_id'].append(doc_id)
    val_dict['orig_quest'].append(quest)
    val_dict['questions'].append(actual_list)
   

Processing Documents: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:20<00:00,  1.19s/it]

Error parsing the string: unterminated string literal (detected at line 6) (<unknown>, line 6)


In [18]:
import pandas as pd

df = pd.DataFrame.from_dict(val_dict)

# Explode the 'questions' column to create a row for each question
df_ground_truth = df.explode('questions')

# Reset the index if needed
df_ground_truth.reset_index(drop=True, inplace=True)

df_ground_truth

,doc_id,orig_quest,questions
0,34cdb806-9fd2-4b74-895a-134bfdb7bcac,What is hypnosis and is it safe?,What exactly is hypnosis and how does it work?
1,34cdb806-9fd2-4b74-895a-134bfdb7bcac,What is hypnosis and is it safe?,Is hypnosis a safe procedure?
2,34cdb806-9fd2-4b74-895a-134bfdb7bcac,What is hypnosis and is it safe?,Can you explain how hypnosis can be used to ma...
3,34cdb806-9fd2-4b74-895a-134bfdb7bcac,What is hypnosis and is it safe?,Are there any risks associated with going into...
4,34cdb806-9fd2-4b74-895a-134bfdb7bcac,What is hypnosis and is it safe?,How is hypnosis similar to day-dreaming and ca...
...,...,...,...
75,ce3a25a9-cf78-44c3-b0dd-48bb8664c974,Q: I feel quite slightly embarrassed to seek h...,I feel quite slightly embarrassed to seek help...
76,ce3a25a9-cf78-44c3-b0dd-48bb8664c974,Q: I feel quite slightly embarrassed to seek h...,Will my concerns and fears about seeking help ...
77,ce3a25a9-cf78-44c3-b0dd-48bb8664c974,Q: I feel quite slightly embarrassed to seek h...,What are some of the barriers that I might pas...
78,ce3a25a9-cf78-44c3-b0dd-48bb8664c974,Q: I feel quite slightly embarrassed to seek h...,Is it common for people to have reservations o...


In [19]:
df_ground_truth.to_csv('ground-truth-data.csv', index=False)